#### Name: Nandakumar Abbigeri
#### Email: officialnandu@gmail.com

## <font color="red">Read these comments before checking through.</font>
* For the sake of visualizing the data in intermediate steps between actions the lazy-evaluation of spark is ignored.
* The answer to each question is at the end code-block of the respective section in this ipnb
* Assumptions are stated as and when required

# <font color="red">Important Assumption</font>
* The given data-set in the learning portal of the s3-bucket for the "parking-violation" dataset is not working.
* So as per my assignment walkthrough, I have uploaded "Parking_Violation_2016" dataset into my account aws S3 bucket and carried out the requirements of the assignment. 

##  <font color="green">Note on color-scheme used
* All the red font text are the questions that are asked in the assignment.
* All the blue font text are stated assumptions or comments on computation steps.
* All the black font text are answers to the questions or some important points/notes to be considered.

In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1668001698833_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## <font color="red">Reading the data</font>

In [2]:
# Note: This is the data from kaggle 2016 data uploaded to my AWS account S3-bucket.
s3path = 's3://upgrad-data/Parking_Violation_Tickets.csv'

df = spark.read.format("csv").option("inferSchema","true").option("header", "true").load(s3path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+------------+-------------------+-------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+--------+---------+---------------+------------------+------------+----+----+----+
|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicl

# <font color="red">Removing all the un-necessary columns from the entire data</font>

In [4]:
considered_columns = {"Summons Number","Plate ID","Registration State","Issue Date","Violation Code","Vehicle Body Type","Vehicle Make", "Violation Precinct","Issue Precinct","Violation Time"}
df = df.select([c for c in df.columns if c in considered_columns])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Violation Time|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+
|    1363745270| GGY6450|                99|07/09/2015|            46|              SDN|       HONDA|                74|         1037A|
|    1363745293|  KXD355|                SC|07/09/2015|            21|             SUBN|       CHEVR|                79|         1206P|
|    1363745438| JCK7576|                PA|07/09/2015|            21|              SDN|       ME/BE|                79|         0820A|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+
only showing top 3 rows

In [6]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10626899

In [7]:
df = df.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10626899

In [9]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Violation Time: string (nullable = true)

# <font color="red">Assignment QnA</font>

# <font color="red">Data Examination</font>

## <font color=red>Q1: Find the total number of tickets for the year.

In [10]:
df = df.dropDuplicates()
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10626899

#### Ans: The total number of tickets for the year 2016 were 10626899

#### <font color="blue">Note: From above analysis performed till now it is clear that there is no duplicate data, i.e each row corresponds unique ticket.</font>

## <font color=red>Q2: Find out the number of unique states from where the cars the got the parking tickets came.

#### <font color="blue">Initial check for unique states</font>

In [11]:
df.createOrReplaceTempView("Vdata")
spark.sql("select count(distinct `Registration State`) as `Registration State`\
          from Vdata").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|Registration State|
+------------------+
|                68|
+------------------+

#### Ans: There are 68 unique states of the required condition without data clean

#### <font color="blue">Data-cleaning for the entry of 99 in "Registration State"</font>

In [12]:
from pyspark.sql.functions import *

# For replacing by the state with max entries, we can replace 99 by mode of the column
mode_value = df.groupby("Registration State").count().orderBy("count", ascending=False).first()[0]
df = df.withColumn('Registration State', when(df['Registration State']=="99",mode_value).otherwise(df["Registration State"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Violation Time|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+
|    1334907778| GHG3401|                NY|06/23/2015|            20|             SUBN|       HONDA|                71|         0644P|
|    1335102358| GWT7769|                NY|06/25/2015|            24|              SDN|       BUICK|                75|         0310A|
|    1259656421| GNA3063|                PA|06/21/2015|            21|             SUBN|       NISSA|                88|         0125A|
|    1259672189| GGT3952|                NY|07/08/2015|            71|             SUBN|       DODGE|                84|         0339P|
|    1364639750| GZJ8497|                NY|07/2

In [14]:
# Recheck for the unique states
spark.sql("select count(distinct 'Registration State') as 'Registration State'\
          from Vdata").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|Registration State|
+------------------+
|                68|
+------------------+

#### Ans: There are 68 unique states of the required condition with data clean

## <font color=red>Q3: Display the top 20 states with the most number of tickets along with their ticket count.

In [15]:
df.groupBy('Registration State').count().orderBy('count', ascending=False).show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------+
|Registration State|  count|
+------------------+-------+
|                NY|8301774|
|                NJ| 968839|
|                PA| 259177|
|                CT| 145153|
|                FL| 138647|
|                MA|  99115|
|                IN|  81141|
|                VA|  75093|
|                MD|  60240|
|                NC|  55629|
|                IL|  37222|
|                GA|  35300|
|                TX|  32635|
|                AZ|  26603|
|                ME|  24277|
|                OH|  23784|
|                CA|  23532|
|                OK|  21782|
|                SC|  21203|
|                TN|  18872|
+------------------+-------+
only showing top 20 rows

# <font color="red">Aggregation tasks</font>

## <font color=red>Q1: How often does each violation code occur? Display the frequency of the top five violation codes.

In [16]:
df.groupBy('Violation Code').count().orderBy('count', ascending=False).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------+
|Violation Code|  count|
+--------------+-------+
|            21|1531587|
|            36|1253512|
|            38|1143696|
|            14| 875614|
|            37| 686610|
+--------------+-------+
only showing top 5 rows

## <font color=red>Q2: How often does each 'vehicle body type' get a parking ticket? How about the 'vehicle make'? Find the top 5 for both.

#### <font color="blue"> For vehicle body type</font>

In [17]:
spark.sql("select 'Vehicle Body Type', count(*) as 'Number of tickets'\
          from Vdata\
          group by 'Vehicle Body Type'\
          order by 'Number of tickets' desc\
          limit 5").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------------+
|Vehicle Body Type|Number of tickets|
+-----------------+-----------------+
|             SUBN|          3466037|
|             4DSD|          2992107|
|              VAN|          1518303|
|             DELV|           755282|
|              SDN|           424043|
+-----------------+-----------------+

#### <font color="blue"> For vehicle make</font>

In [18]:
spark.sql("select 'Vehicle Make', count(*) as 'Number of tickets'\
          from Vdata\
          group by 'Vehicle Make'\
          order by 'Number of tickets' desc\
          limit 5").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------------+
|Vehicle Make|Number of tickets|
+------------+-----------------+
|        FORD|          1324774|
|       TOYOT|          1154790|
|       HONDA|          1014074|
|       NISSA|           834833|
|       CHEVR|           759663|
+------------+-----------------+

## <font color=red>Q3: Seasonality in data.

### <font color="blue">Assumption: The given dateformat for column "Issue Date" for this question is assumed to be "dd/mm/yyyy"</font>

In [19]:
df.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Violation Time|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+
|    1334907778| GHG3401|                NY|06/23/2015|            20|             SUBN|       HONDA|                71|         0644P|
|    1335102358| GWT7769|                NY|06/25/2015|            24|              SDN|       BUICK|                75|         0310A|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+
only showing top 2 rows

####  <font color="blue">Recast the "Issue Date" column to proper format</font>

In [20]:
pattern = 'dd/MM/yyyy'
df = df.withColumn("Issue Date",to_date(col("Issue Date"),pattern))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Violation Time|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+
|    1334907778| GHG3401|                NY|      null|            20|             SUBN|       HONDA|                71|         0644P|
|    1335102358| GWT7769|                NY|      null|            24|              SDN|       BUICK|                75|         0310A|
|    1259656421| GNA3063|                PA|      null|            21|             SUBN|       NISSA|                88|         0125A|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+
only showing top 3 rows

In [22]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

def get_season(date):
    # Assumption: We assign a value of "Season-0" for all the null values in the "Issue Date" in "Season" column
    if(date==None):
        return "Season-0"
    SeasonMap = {
    1: "Season-1",
    2: "Season-1",
    3: "Season-1",
    4: "Season-2",
    5: "Season-2",
    6: "Season-2",
    7: "Season-3",
    8: "Season-3",
    9: "Season-3",
    10: "Season-4",
    11: "Season-4",
    12: "Season-4",
    }
    return SeasonMap.get(int(date.month))

get_season = udf(get_season, StringType())

df = df.withColumn('Season', get_season(df['Issue Date']))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+--------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Violation Time|  Season|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+--------+
|    1334907778| GHG3401|                NY|      null|            20|             SUBN|       HONDA|                71|         0644P|Season-0|
|    1335102358| GWT7769|                NY|      null|            24|              SDN|       BUICK|                75|         0310A|Season-0|
|    1259656421| GNA3063|                PA|      null|            21|             SUBN|       NISSA|                88|         0125A|Season-0|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+----------

In [24]:
df.groupBy('Issue Date').count().orderBy('count', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+
|Issue Date|  count|
+----------+-------+
|      null|6369221|
|2015-01-10|  47864|
|2015-08-10|  47649|
|2015-07-10|  45145|
|2016-01-03|  45087|
|2015-06-11|  45000|
|2015-11-09|  44870|
|2015-02-11|  44455|
|2016-08-03|  44165|
|2015-09-10|  44120|
|2016-02-02|  43656|
|2015-09-11|  43583|
|2015-06-10|  43575|
|2016-04-02|  43341|
|2015-04-11|  43323|
|2015-05-11|  43279|
|2015-12-11|  43159|
|2016-10-03|  42875|
|2016-10-05|  42813|
|2016-12-05|  42111|
+----------+-------+
only showing top 20 rows

#### <font color="blue">Note: From the null value count above it is clear that, there are entries with no "Issue Date"
<font color=blue>* So in order to cater for these entries we assign these with Season of "Season-0" as shown below</font>

In [25]:
df.groupBy('Season').count().orderBy('count', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+
|  Season|  count|
+--------+-------+
|Season-0|6369221|
|Season-3|1111511|
|Season-4|1072349|
|Season-2|1041412|
|Season-1|1032406|
+--------+-------+

In [26]:
df2 = df.groupBy("Season","Violation Code").count().orderBy("count",ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------+------+
|  Season|Violation Code| count|
+--------+--------------+------+
|Season-0|            21|934298|
|Season-0|            36|727336|
|Season-0|            38|707333|
|Season-0|            14|535322|
|Season-0|            37|420450|
|Season-0|            20|376546|
|Season-0|            46|355491|
|Season-0|             7|308703|
|Season-0|            40|282535|
|Season-0|            71|200927|
|Season-0|            19|178094|
|Season-3|            21|158135|
|Season-1|            21|157935|
|Season-4|            21|155045|
|Season-3|            36|147961|
|Season-0|            69|139507|
|Season-2|            36|137980|
|Season-0|            70|136864|
|Season-4|            36|135186|
|Season-2|            21|126174|
+--------+--------------+------+
only showing top 20 rows

In [28]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

windowDept = Window.partitionBy("Season").orderBy(col("count").desc())

df3 = df2.withColumn("row",row_number().over(windowDept)).filter(col("row") <= 3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df3.orderBy("Season").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------+------+---+
|  Season|Violation Code| count|row|
+--------+--------------+------+---+
|Season-0|            21|934298|  1|
|Season-0|            36|727336|  2|
|Season-0|            38|707333|  3|
|Season-1|            36|105049|  2|
|Season-1|            38|101428|  3|
|Season-1|            21|157935|  1|
|Season-2|            36|137980|  1|
|Season-2|            21|126174|  2|
|Season-2|            38|107123|  3|
|Season-3|            21|158135|  1|
|Season-3|            36|147961|  2|
|Season-3|            38|111787|  3|
|Season-4|            21|155045|  1|
|Season-4|            36|135186|  2|
|Season-4|            38|116025|  3|
+--------+--------------+------+---+

#### Ans: The above table gives the seasonal violation code counts for top 3 violation codes
##### Note: Here "Season-0" refers to data-entries with null-type for "Issue Date"

## <font color=red>Q4: Revenue from parking violations

### <font color=red>Total occurences of the three of the most common violation codes.

In [31]:
df_temp = (df.groupBy('Violation Code').count().orderBy('count', ascending=False)).head(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df_temp = spark.createDataFrame(df_temp)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_temp.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------+
|Violation Code|  count|
+--------------+-------+
|            21|1531587|
|            36|1253512|
|            38|1143696|
+--------------+-------+

### <font color=red>Total amount collected for the above common codes

In [34]:
# This is collected from nyc-website with averaging over region as specified
violation_fine={ 21:65, 36:50, 38:100,}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
def get_revenue(violation_code, count):
    violation_fine={ 21:65, 36:50, 38:100,} # This data is from the given website
    return (violation_fine.get(violation_code)*count)

get_revenue = udf(get_revenue, LongType())

df_temp = df_temp.withColumn('Total Revenue in $', get_revenue(df_temp['Violation Code'],df_temp['count']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
df_temp.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------+------------------+
|Violation Code|  count|Total Revenue in $|
+--------------+-------+------------------+
|            21|1531587|          99553155|
|            36|1253512|          62675600|
|            38|1143696|         114369600|
+--------------+-------+------------------+

##### Note: The code with highest collection is 38 with a collection of \$114369600

### <font color=red>Top 3 states that have the highest ticket revenue based on the above top 3 violation codes alone

In [38]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+--------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Violation Time|  Season|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+--------+
|    1334907778| GHG3401|                NY|      null|            20|             SUBN|       HONDA|                71|         0644P|Season-0|
|    1335102358| GWT7769|                NY|      null|            24|              SDN|       BUICK|                75|         0310A|Season-0|
|    1259656421| GNA3063|                PA|      null|            21|             SUBN|       NISSA|                88|         0125A|Season-0|
|    1259672189| GGT3952|                NY|2015-08-07|            71|             SUBN|       DODGE|                84|         0

In [39]:
def get_revenue(violation_code):
    
    violation_fine={ 21:65, 36:50, 38:100,} # This data is from the given website
    
    if(violation_fine.get(violation_code)==None): # Note: Temporarily returning 0 for non significant fines
        return 0
    else:
        return (violation_fine.get(violation_code))

get_revenue = udf(get_revenue, LongType())

df = df.withColumn('Total Revenue in $', get_revenue(df['Violation Code']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+--------+------------------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Violation Time|  Season|Total Revenue in $|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+--------------+--------+------------------+
|    1334907778| GHG3401|                NY|      null|            20|             SUBN|       HONDA|                71|         0644P|Season-0|                 0|
|    1335102358| GWT7769|                NY|      null|            24|              SDN|       BUICK|                75|         0310A|Season-0|                 0|
|    1259656421| GNA3063|                PA|      null|            21|             SUBN|       NISSA|                88|         0125A|Season-0|                65|
+--------------+

In [41]:
df_temp = df.groupBy('Registration State').sum('Total Revenue in $')
df_temp.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------------+
|Registration State|sum(Total Revenue in $)|
+------------------+-----------------------+
|                MN|                 209115|
|                DC|                 136760|
|                MX|                    195|
|                GV|                   2080|
|                QB|                 147360|
|                MD|                1877995|
|                DE|                 524035|
|                MO|                 101550|
|                IL|                 893375|
|                MS|                 135395|
|                SD|                  75190|
|                ON|                 153485|
|                AK|                  32145|
|                UT|                  55840|
|                HI|                  22705|
|                NS|                  29025|
|                CA|                 794135|
|                CT|                4291305|
|                NC|                1707690|
|         

In [42]:
df_temp = df_temp.orderBy('sum(Total Revenue in $)', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
df_temp.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------------+
|Registration State|sum(Total Revenue in $)|
+------------------+-----------------------+
|                NY|              218585075|
|                NJ|               20488260|
|                PA|                7749500|
+------------------+-----------------------+
only showing top 3 rows

##### The above are the top three state represented by their registration codes

## <font color=red>Few Important Deductions from above findings

##### Note here "NY: New York", "NJ: New Jersey", "PA: Pennsylvania "

 * Cars that have registration from NY aggregate to form the highest revenue just from the above violation codes alone (21,36,38)  followed by NJ and PA.
 
* It is surprising to note that there is no revenue from the cars of the state NT for above three violation codes.
